In [12]:
from tensorflow.keras.models import model_from_json
import numpy as np
import os
from music21 import *
import sys
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")
np.random.seed(42)
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline
from collections import Counter

In [13]:
# Load the model architecture from JSON file
with open('model_architecture.json', 'r') as json_file:
    loaded_model_architecture = json_file.read()

loaded_model = model_from_json(loaded_model_architecture)

# Load the trained weights
loaded_model.load_weights('final_model_weights.h5')

# Load X_seed from your saved data
# Example: Assuming X_seed.npy is saved
X_seed = np.load('X_seed.npy')

length = 40


In [8]:
#Loading the list of chopin's midi files as stream
filepath = "classical-music-midi/chopin/"
#Getting midi files
all_midis= []
for i in os.listdir(filepath):
    if i.endswith(".mid"):
        tr = filepath+i
        midi = converter.parse(tr)
        all_midis.append(midi)

In [9]:
#Helping function
def extract_notes(file):
    notes = []
    pick = None
    for j in file:
        songs = instrument.partitionByInstrument(j)
        for part in songs.parts:
            pick = part.recurse()
            for element in pick:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append(".".join(str(n) for n in element.normalOrder))

    return notes

In [10]:
#Getting the list of notes as Corpus
Corpus= extract_notes(all_midis)
print("Total notes in all the Chopin midis in the dataset:", len(Corpus))

Total notes in all the Chopin midis in the dataset: 57894


In [14]:
count_num = Counter(Corpus)

In [15]:
#Exploring the notes dictionary
Notes = list(count_num.keys())
Recurrence = list(count_num.values())
#Average recurrenc for a note in Corpus
def Average(lst):
    return sum(lst) / len(lst)
print("Average recurrenc for a note in Corpus:", Average(Recurrence))
print("Most frequent note in Corpus appeared:", max(Recurrence), "times")
print("Least frequent note in Corpus appeared:", min(Recurrence), "time")

Average recurrenc for a note in Corpus: 145.8287153652393
Most frequent note in Corpus appeared: 1627 times
Least frequent note in Corpus appeared: 1 time


In [16]:
#Getting a list of rare chords
rare_note = []
for index, (key, value) in enumerate(count_num.items()):
    if value < 100:
        m =  key
        rare_note.append(m)

print("Total number of notes that occur less than 100 times:", len(rare_note))


Total number of notes that occur less than 100 times: 270


In [17]:
#Eleminating the rare notes
for element in Corpus:
    if element in rare_note:
        Corpus.remove(element)

print("Length of Corpus after elemination the rare notes:", len(Corpus))

Length of Corpus after elemination the rare notes: 53753


In [18]:
# Storing all the unique characters present in my corpus to bult a mapping dic.
symb = sorted(list(set(Corpus)))

L_corpus = len(Corpus) #length of corpus
L_symb = len(symb) #length of total unique characters

#Building dictionary to access the vocabulary from indices and vice versa
mapping = dict((c, i) for i, c in enumerate(symb))
reverse_mapping = dict((i, c) for i, c in enumerate(symb))

print("Total number of characters:", L_corpus)
print("Number of unique characters:", L_symb)

Total number of characters: 53753
Number of unique characters: 263


In [19]:
def chords_n_notes(Snippet):
    Melody = []
    offset = 0 #Incremental
    for i in Snippet:
        #If it is chord
        if ("." in i or i.isdigit()):
            chord_notes = i.split(".") #Seperating the notes in chord
            notes = []
            for j in chord_notes:
                inst_note=int(j)
                note_snip = note.Note(inst_note)
                notes.append(note_snip)
                chord_snip = chord.Chord(notes)
                chord_snip.offset = offset
                Melody.append(chord_snip)
        # pattern is a note
        else:
            note_snip = note.Note(i)
            note_snip.offset = offset
            Melody.append(note_snip)
        # increase offset each iteration so that notes do not stack
        offset += 1
    Melody_midi = stream.Stream(Melody)
    return Melody_midi

In [29]:
def Malody_Generator(model, X_seed, Note_Count):
    seed = X_seed[np.random.randint(0, len(X_seed)-1)]
    Music = ""
    Notes_Generated = []
    for i in range(Note_Count):
        seed = seed.reshape(1, length, 1)
        prediction = model.predict(seed, verbose=0)[0]
        prediction = np.log(prediction) / 1.0  # diversity
        exp_preds = np.exp(prediction)
        prediction = exp_preds / np.sum(exp_preds)
        index = np.argmax(prediction)
        index_N = index / float(L_symb)
        Notes_Generated.append(index)        
        Music = [reverse_mapping[char] for char in Notes_Generated]
        seed = np.insert(seed[0], len(seed[0]), index_N)
        seed = seed[1:]
    Melody = chords_n_notes(Music)
    Melody_midi = stream.Stream(Melody)
    return Music, Melody_midi





In [24]:
# Generate music
Music_notes, Melody = Malody_Generator(loaded_model, X_seed, 200)
# Save generated melody to MIDI file
Melody.write('midi', 'mid/Melody_Generated.mid')

[221]
[221, 196]
[221, 196, 208]
[221, 196, 208, 112]
[221, 196, 208, 112, 222]
[221, 196, 208, 112, 222, 48]
[221, 196, 208, 112, 222, 48, 233]
[221, 196, 208, 112, 222, 48, 233, 221]
[221, 196, 208, 112, 222, 48, 233, 221, 196]
[221, 196, 208, 112, 222, 48, 233, 221, 196, 89]
[221, 196, 208, 112, 222, 48, 233, 221, 196, 89, 247]
[221, 196, 208, 112, 222, 48, 233, 221, 196, 89, 247, 119]
[221, 196, 208, 112, 222, 48, 233, 221, 196, 89, 247, 119, 240]
[221, 196, 208, 112, 222, 48, 233, 221, 196, 89, 247, 119, 240, 196]
[221, 196, 208, 112, 222, 48, 233, 221, 196, 89, 247, 119, 240, 196, 27]
[221, 196, 208, 112, 222, 48, 233, 221, 196, 89, 247, 119, 240, 196, 27, 196]
[221, 196, 208, 112, 222, 48, 233, 221, 196, 89, 247, 119, 240, 196, 27, 196, 201]
[221, 196, 208, 112, 222, 48, 233, 221, 196, 89, 247, 119, 240, 196, 27, 196, 201, 241]
[221, 196, 208, 112, 222, 48, 233, 221, 196, 89, 247, 119, 240, 196, 27, 196, 201, 241, 174]
[221, 196, 208, 112, 222, 48, 233, 221, 196, 89, 247, 119, 2

'mid/Melody_Generated.mid'